In [1]:
import pandas as pd
import numpy as np
import string
from collections import Counter
from utils import *
from vehicle import Vehicle

import networkx as nx
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx


## Estrazione dati con un piccolo parser dei dati

In [2]:
path='Instances/N6.txt'

In [3]:
#support
count = 0
#first 3 lines of the txt file
number_of_customers=0
one=0
number_of_vehicles=0
#vettori di informazione
vect_cord=[] #lista/vettore di coppiadi coordinate
vect_domanda=[] #linehaul
vect_carico=[] #backhaul
vec_b=[] #vettore delle istanze backhaul
vec_l=[] #vettore delle istanze linehaul
with open(path) as file:
    for line in file:
        #print(line)
        count+=1
        if count  == 1:
            number_of_customers=int((line.split())[0])
        if count  == 2:
            one=int((line.split())[0])
        if count == 3:
            number_of_vehicles=int((line.split())[0])
        if count > 3:
            vect_cord.append((int((line.split())[0]),int((line.split())[1])))
            vect_domanda.append((int((line.split())[2])))
            vect_carico.append((int((line.split())[3]))) 
            if int((line.split())[3]) == 0:
                vec_l.append(count-4) #il numero delle istanze linehaul
            if int((line.split())[3]) > 0:
                vec_b.append(count-4) # il numero delle istanze backhaul

#capacità massima di ogni mezzo
capacity=vect_carico[0]                     

In [4]:
df_data = pd.DataFrame({'': [number_of_customers, number_of_vehicles, capacity]},
                       index=['Number of customers', 'Number of Vehicles', 'Capacity'])

In [5]:
df_instances = pd.DataFrame({'Cordinate X': [ x[0] for x in vect_cord],
                             'Cordinate Y': [ x[1] for x in vect_cord], 
                             'Linehaul': vect_domanda, 
                             'Backhaul' : vect_carico,
                            })

In [6]:
df_data

,
Number of customers,150
Number of Vehicles,8
Capacity,8500


all'indice 0 è riportato il magazzino. dove il campo Backhaul rappresenta la capacità massima

In [7]:
df_instances

,Cordinate X,Cordinate Y,Linehaul,Backhaul
0,12000,16000,0,8500
1,1725,21949,0,820
2,22443,15651,0,291
3,11622,7533,0,344
4,14613,7181,0,244
5,771,10569,0,714
6,18508,15099,0,311
7,16029,5701,0,267
8,20666,29528,0,638
9,7898,29590,0,362


In [8]:
dict_coord = {}
for index, value in enumerate(vect_cord):
    dict_coord[index]=(value[0]/10000, value[1]/10000)

In [9]:
dict_coord

{0: (1.2, 1.6),
 1: (0.1725, 2.1949),
 2: (2.2443, 1.5651),
 3: (1.1622, 0.7533),
 4: (1.4613, 0.7181),
 5: (0.0771, 1.0569),
 6: (1.8508, 1.5099),
 7: (1.6029, 0.5701),
 8: (2.0666, 2.9528),
 9: (0.7898, 2.959),
 10: (0.7557, 1.1209),
 11: (2.2533, 2.7056),
 12: (1.3861, 1.0795),
 13: (1.0578, 0.3881),
 14: (0.1625, 1.9201),
 15: (1.8608, 2.6115),
 16: (0.567, 2.6554),
 17: (2.1409, 0.9013),
 18: (1.2465, 2.3593),
 19: (1.6852, 1.8097),
 20: (0.841, 0.1322),
 21: (0.3863, 2.9503),
 22: (0.4114, 0.9954),
 23: (0.4736, 1.8223),
 24: (0.9835, 1.0318),
 25: (1.8257, 2.0218),
 26: (1.2606, 2.3154),
 27: (1.8578, 1.136),
 28: (2.2262, 0.8762),
 29: (0.4626, 2.673),
 30: (0.1976, 1.806),
 31: (2.2413, 1.2376),
 32: (0.9725, 1.1171),
 33: (2.2724, 0.3555),
 34: (0.826, 0.954),
 35: (0.3994, 0.614),
 36: (0.7979, 0.3116),
 37: (1.2867, 2.5914),
 38: (1.9211, 1.2464),
 39: (2.3637, 0.8223),
 40: (0.5058, 2.8762),
 41: (1.0809, 1.8674),
 42: (2.3427, 1.2401),
 43: (2.0817, 0.3141),
 44: (1.2997,

In [10]:
import networkx as nx
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4
from bokeh.io import output_notebook
G=nx.Graph()
for e in list(dict_coord.keys()):
    G.add_node(e,pos=dict_coord[e])

In [11]:
pos=nx.get_node_attributes(G,'pos')

In [12]:
plot = Plot(plot_width=500, plot_height=500,
            x_range=Range1d(-1.3, 3.3), y_range=Range1d(-1.3, 3.3))
plot.title.text = "Graph Interaction"

node_hover_tool = HoverTool(tooltips=[("Magazzino", "@index"),("Coordinate", '@pos')])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, pos, scale=1, center=(0, 0))
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine( line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

output_notebook()
show(plot)

Loading BokehJS ...

## Calcolo dei Dati raccolti

In [13]:
distance = distance_matrix(vect_cord)

In [14]:
dict_distance={}
for c, i in enumerate(distance):
    for k,j in enumerate(i):
        dict_distance[(c,k)]=j

In [15]:
dict_distance

{(0, 0): 0.0,
 (0, 1): 11872.919859916516,
 (0, 2): 10448.8300780518,
 (0, 3): 8475.433499237664,
 (0, 4): 9197.963361527376,
 (0, 5): 12473.419819760738,
 (0, 6): 6570.073439467781,
 (0, 7): 11059.034406312334,
 (0, 8): 16065.688282796975,
 (0, 9): 14195.580438995794,
 (0, 10): 6534.059228381696,
 (0, 11): 15270.207104031038,
 (0, 12): 5527.689028879971,
 (0, 13): 12202.141000660498,
 (0, 14): 10857.579196119179,
 (0, 15): 12082.172362617577,
 (0, 16): 12306.738641898592,
 (0, 17): 11719.532840518857,
 (0, 18): 7607.225118267502,
 (0, 19): 5285.765129099098,
 (0, 20): 15110.651342678779,
 (0, 21): 15765.207832439128,
 (0, 22): 9936.956878239936,
 (0, 23): 7596.540330966459,
 (0, 24): 6080.489207292453,
 (0, 25): 7545.964020587429,
 (0, 26): 7179.620602789538,
 (0, 27): 8049.825091267511,
 (0, 28): 12557.758080167017,
 (0, 29): 13019.553602178532,
 (0, 30): 10233.483082509103,
 (0, 31): 11025.604065084144,
 (0, 32): 5338.058261203225,
 (0, 33): 16428.091824676412,
 (0, 34): 7464.529456

In [16]:
from collections import Counter
c = Counter(dict_distance) #inizializzo una variabile della classe Counter 
dict_distance_Order=c.most_common() #ordino per i valori dei saving in maniera decrescente
dict_distance = {} # svuoto il vecchio dizionario per questioni di memoria
for e in dict_distance_Order: # ciclo su ogni valore del vettore della classe counter che è composto da copie 
    dict_distance[e[0]]=e[1] # reinserisco su K il nome delle istanze e su V il valore a loro associato
   

In [17]:
for e in dict_distance:
    if 0 in e:
        print(e)

(0, 117)
(117, 0)
(0, 149)
(149, 0)
(0, 33)
(33, 0)
(0, 8)
(8, 0)
(0, 150)
(150, 0)
(0, 21)
(21, 0)
(0, 43)
(43, 0)
(0, 144)
(144, 0)
(0, 85)
(85, 0)
(0, 64)
(64, 0)
(0, 11)
(11, 0)
(0, 20)
(20, 0)
(0, 48)
(48, 0)
(0, 93)
(93, 0)
(0, 113)
(113, 0)
(0, 40)
(40, 0)
(0, 77)
(77, 0)
(0, 88)
(88, 0)
(0, 9)
(9, 0)
(0, 68)
(68, 0)
(0, 79)
(79, 0)
(0, 39)
(39, 0)
(0, 131)
(131, 0)
(0, 148)
(148, 0)
(0, 69)
(69, 0)
(0, 53)
(53, 0)
(0, 121)
(121, 0)
(0, 50)
(50, 0)
(0, 61)
(61, 0)
(0, 36)
(36, 0)
(0, 71)
(71, 0)
(0, 45)
(45, 0)
(0, 29)
(29, 0)
(0, 118)
(118, 0)
(0, 95)
(95, 0)
(0, 86)
(86, 0)
(0, 35)
(35, 0)
(0, 114)
(114, 0)
(0, 28)
(28, 0)
(0, 137)
(137, 0)
(0, 5)
(5, 0)
(0, 92)
(92, 0)
(0, 16)
(16, 0)
(0, 70)
(70, 0)
(0, 13)
(13, 0)
(0, 134)
(134, 0)
(0, 147)
(147, 0)
(0, 15)
(15, 0)
(0, 42)
(42, 0)
(0, 75)
(75, 0)
(0, 49)
(49, 0)
(0, 1)
(1, 0)
(0, 63)
(63, 0)
(0, 99)
(99, 0)
(0, 123)
(123, 0)
(0, 112)
(112, 0)
(0, 106)
(106, 0)
(0, 17)
(17, 0)
(0, 122)
(122, 0)
(0, 130)
(130, 0)
(0, 67)
(67,

In [18]:
saving=saving_matrix(distance)

In [19]:
dict_Saving_Order=savingOrder(saving)

## Mappa dei saving ordinati

In [20]:
dict_b, dict_l , dict_merge= route_Linehaul_Backhaul(dict_Saving_Order, vec_b, vec_l)

In [21]:
dict_l

{(117, 149): 34261.807736035546,
 (64, 144): 30555.23299547494,
 (117, 144): 29191.93595353028,
 (64, 117): 28911.909544612186,
 (113, 117): 28810.021958485915,
 (144, 150): 28519.04686266807,
 (64, 150): 28423.94575282535,
 (68, 85): 28285.37424885219,
 (113, 144): 28183.432724721606,
 (64, 113): 28049.18693147097,
 (113, 149): 28013.760328614262,
 (144, 149): 27771.399276754586,
 (64, 149): 27526.304114048176,
 (53, 85): 27415.837968900414,
 (117, 150): 27254.130371714098,
 (61, 77): 26825.564521178836,
 (53, 68): 26819.32807997245,
 (93, 149): 26702.19690386529,
 (93, 117): 26133.624812503873,
 (113, 150): 25956.36509563391,
 (77, 118): 25885.089423731024,
 (85, 88): 25820.309187118426,
 (61, 118): 25791.8456364086,
 (149, 150): 25533.20230488049,
 (53, 88): 25045.566244516347,
 (86, 88): 24985.468651580133,
 (77, 131): 24669.69638565308,
 (85, 86): 24664.54596763256,
 (53, 86): 24648.58142075183,
 (61, 131): 24643.41517879674,
 (69, 95): 24598.32150584749,
 (121, 148): 24538.589959

## Inizializzo la lista delle rotte

In [22]:
list_route = []
set_iniziale = {0}
set_first = {0}
set_visited = {0}
lista_keys= list(dict_l.keys())
for k in lista_keys:
    if len(list_route) < number_of_vehicles and k[0] not in set_iniziale and k[1] not in set_iniziale:
        list_route.append(Vehicle([0, k[0], k[1]], vect_domanda[k[0]]+vect_domanda[k[1]],
                         distance[0][k[0]]+distance[k[0]][k[1]]))
        set_iniziale.add(k[0])
        set_iniziale.add(k[1])
        set_first.add(k[0])
        set_visited.add(k[0])
        set_visited.add(k[1])

In [23]:
remove_first_instance(dict_l, set_first)

In [24]:
for e in list_route:
    print(e.route)

[0, 117, 149]
[0, 64, 144]
[0, 68, 85]
[0, 61, 77]
[0, 113, 150]
[0, 53, 88]
[0, 69, 95]
[0, 121, 148]


In [25]:
dict_l

{(144, 150): 28519.04686266807,
 (144, 149): 27771.399276754586,
 (93, 149): 26702.19690386529,
 (77, 118): 25885.089423731024,
 (85, 88): 25820.309187118426,
 (149, 150): 25533.20230488049,
 (86, 88): 24985.468651580133,
 (77, 131): 24669.69638565308,
 (85, 86): 24664.54596763256,
 (70, 79): 24251.59876063198,
 (71, 148): 24235.20247158954,
 (147, 150): 23899.719952219384,
 (63, 79): 23744.74903208721,
 (99, 147): 23708.10682350985,
 (118, 131): 23560.839067826983,
 (99, 150): 23509.77711969217,
 (79, 150): 23429.713551553956,
 (71, 112): 23345.579148951612,
 (93, 130): 23280.79162361122,
 (88, 137): 23109.70641298176,
 (63, 70): 23075.162710125896,
 (120, 149): 23040.511948264244,
 (95, 134): 23039.951833862877,
 (106, 122): 22957.290059810715,
 (71, 85): 22954.724629405984,
 (92, 133): 22913.790112505794,
 (75, 93): 22873.686914692546,
 (85, 90): 22844.028979117553,
 (146, 147): 22735.53920374217,
 (67, 75): 22651.49366009242,
 (97, 149): 22566.57718021349,
 (93, 98): 22559.90576132

In [26]:
for e in dict_l:
    if 9 in e:
        print(e)

In [27]:
a=vect_domanda[10]+vect_domanda[12]+vect_domanda[14]+vect_domanda[21]+vect_domanda[16]
b=vect_domanda[19]+vect_domanda[23]+vect_domanda[6]+vect_domanda[7]+vect_domanda[18]
c=vect_domanda[13]+vect_domanda[25]+vect_domanda[8]+vect_domanda[15]+vect_domanda[11]
d=vect_domanda[20]+vect_domanda[22]+vect_domanda[24]+vect_domanda[9]+vect_domanda[17]

In [28]:
print(a)
print(b)
print(c)
print(d)

0
0
0
0


In [29]:
c = 0
temp = dict_l.copy()
set_first = {0}
i = 0 
while(i < np.math.factorial(number_of_customers)/(np.math.factorial(number_of_customers-2)*2)):
    lista_keys = list(dict_l.keys())
    flag = True
    for k in lista_keys:
        #if 17 in k and list_route[c].route[-1] in k: #controllo cosa c'è quando arriva a 17
            #print(f'La route {c}: \t\t {list_route[c].route} \t\t capacità raggiunta: {list_route[c].weight} \t\t pair {k}')
            #print('----')
        if list_route[c].route[-1] in k \
        and different(list_route[c].route[-1], k) not in set_visited \
        and list_route[c].weight + vect_domanda[different(list_route[c].route[-1],k)] <= capacity:
                
                list_route[c].add_weight(vect_domanda[different(list_route[c].route[-1],k)])
                list_route[c].add_route(k)           
                set_visited.add(list_route[c].route[-1])
                list_route[c].add_cost(distance[k[0]][k[1]])
                set_first.add(k[0])

                break

    #remove_first_instance(dict_l, set_first)
    c = (c + 1) % number_of_vehicles
    
    i += 1

In [30]:
len(dict_l)

4186

In [31]:
for e in list_route:
    e.delivery = e.weight
    e.weight = 0
    print(e.route)

[0, 117, 149, 93, 130, 98, 75, 67, 97, 120, 126, 116, 139, 125]
[0, 64, 144, 147, 146, 79, 70, 91, 65, 74, 51, 89, 104, 141]
[0, 68, 85, 86, 90, 138, 101, 108, 59, 80, 135, 55, 57, 105]
[0, 61, 77, 118, 131, 123, 114, 102, 96, 78, 72, 83, 62, 87]
[0, 113, 150, 99, 58, 63, 100, 76, 142, 129, 82, 140, 110]
[0, 53, 88, 137, 106, 122, 145, 66, 128, 136, 143, 127, 124]
[0, 69, 95, 134, 103, 92, 133, 56, 73, 107, 60, 119, 84]
[0, 121, 148, 71, 112, 94, 115, 109, 54, 111, 81, 52, 132]


In [33]:
for e in dict_merge:
    if 17 in e:
        print(e)

(17, 95)
(17, 69)
(17, 134)
(17, 103)
(17, 92)
(17, 65)
(17, 133)
(17, 73)
(17, 91)
(17, 100)
(17, 70)
(17, 79)
(17, 76)
(17, 56)
(17, 63)
(17, 74)
(17, 107)
(17, 142)
(17, 51)
(17, 146)
(17, 72)
(17, 129)
(17, 150)
(17, 114)
(17, 58)
(17, 147)
(17, 99)
(17, 78)
(17, 60)
(17, 82)
(17, 64)
(17, 144)
(17, 123)
(17, 116)
(17, 113)
(17, 117)
(17, 89)
(17, 131)
(17, 149)
(17, 126)
(17, 102)
(17, 119)
(17, 120)
(17, 96)
(17, 61)
(17, 140)
(17, 77)
(17, 118)
(17, 104)
(17, 139)
(17, 97)
(17, 136)
(17, 93)
(17, 143)
(17, 130)
(17, 98)
(17, 83)
(17, 128)
(17, 121)
(17, 59)
(17, 75)
(17, 66)
(17, 127)
(17, 67)
(17, 110)
(17, 115)
(17, 84)
(17, 124)
(17, 108)
(17, 148)
(17, 109)
(17, 101)
(17, 132)
(17, 62)
(17, 94)
(17, 71)
(17, 54)
(17, 145)
(17, 112)
(17, 135)
(17, 122)
(17, 111)
(17, 55)
(17, 106)
(17, 80)
(17, 125)
(17, 105)
(17, 90)
(17, 87)
(17, 141)
(17, 138)
(17, 68)
(17, 85)
(17, 53)
(17, 137)
(17, 81)
(17, 86)
(17, 57)
(17, 52)
(17, 88)


In [34]:
a=vect_carico[4]+vect_carico[5]
b=vect_carico[3]
c=vect_carico[2]
d=vect_carico[1]

In [35]:
print(a)
print(b)
print(c)
print(d)

958
344
291
820


In [36]:
c = 0

while(c < len(list_route)):
    if len(set_visited) <= number_of_customers:
        merge = list(dict_merge.keys())
        back = list(dict_b.keys())
        for k in merge:
            if list_route[c].route[-1] in k and different(list_route[c].route[-1], k) not in set_visited:
                    list_route[c].add_weight(vect_carico[different(list_route[c].route[-1],k)])
                    list_route[c].add_route(k)
                    set_visited.add(list_route[c].route[-1])
                    list_route[c].add_cost(distance[k[0]][k[1]]) 
                    break
    c += 1

In [37]:

c = 0
temp = dict_b.copy()
set_first = {0}
i = 0 
while(i < np.math.factorial(number_of_customers)/(np.math.factorial(number_of_customers-2)*2)):
    lista_keys = list(dict_b.keys())
    flag = True
    for k in lista_keys:
        if list_route[c].route[-1] in k \
        and different(list_route[c].route[-1], k) not in set_visited \
        and list_route[c].weight + vect_carico[different(list_route[c].route[-1],k)] <= capacity:
                
                list_route[c].add_weight(vect_carico[different(list_route[c].route[-1],k)])
                list_route[c].add_route(k)           
                set_visited.add(list_route[c].route[-1])
                list_route[c].add_cost(distance[k[0]][k[1]])
                set_first.add(k[0])
                break

    #remove_first_instance(dict_l, set_first)
    c = (c + 1) % number_of_vehicles
    
    i += 1

In [38]:
for e in list_route:
    e.route.append(0)
    e.cost += distance[0][e.route[-1]]
    e.pickUp= e.weight

In [39]:
for e in list_route:
    print(e.route)

[0, 117, 149, 93, 130, 98, 75, 67, 97, 120, 126, 116, 139, 125, 30, 1, 45, 29, 16, 46, 18, 0]
[0, 64, 144, 147, 146, 79, 70, 91, 65, 74, 51, 89, 104, 141, 14, 21, 40, 9, 37, 8, 41, 0]
[0, 68, 85, 86, 90, 138, 101, 108, 59, 80, 135, 55, 57, 105, 5, 36, 13, 4, 27, 11, 0]
[0, 61, 77, 118, 131, 123, 114, 102, 96, 78, 72, 83, 62, 87, 44, 48, 43, 39, 31, 25, 0]
[0, 113, 150, 99, 58, 63, 100, 76, 142, 129, 82, 140, 110, 22, 20, 7, 28, 2, 15, 0]
[0, 53, 88, 137, 106, 122, 145, 66, 128, 136, 143, 127, 124, 35, 47, 3, 17, 38, 19, 0]
[0, 69, 95, 134, 103, 92, 133, 56, 73, 107, 60, 119, 84, 10, 34, 24, 32, 12, 23, 0]
[0, 121, 148, 71, 112, 94, 115, 109, 54, 111, 81, 52, 132, 49, 50, 33, 42, 6, 26, 0]


In [40]:
for e in list_route:
    print(f'Mezzo {list_route.index(e)+1}:\t Rotta: {e.route}\t\t Costo: {e.cost}\t Delivery: {e.delivery}\t Pickup: {e.pickUp}')

Mezzo 1:	 Rotta: [0, 117, 149, 93, 130, 98, 75, 67, 97, 120, 126, 116, 139, 125, 30, 1, 45, 29, 16, 46, 18, 0]		 Costo: 76715.04305386834	 Delivery: 5908	 Pickup: 2871
Mezzo 2:	 Rotta: [0, 64, 144, 147, 146, 79, 70, 91, 65, 74, 51, 89, 104, 141, 14, 21, 40, 9, 37, 8, 41, 0]		 Costo: 103198.398802988	 Delivery: 8348	 Pickup: 3234
Mezzo 3:	 Rotta: [0, 68, 85, 86, 90, 138, 101, 108, 59, 80, 135, 55, 57, 105, 5, 36, 13, 4, 27, 11, 0]		 Costo: 97990.65874518259	 Delivery: 5756	 Pickup: 2787
Mezzo 4:	 Rotta: [0, 61, 77, 118, 131, 123, 114, 102, 96, 78, 72, 83, 62, 87, 44, 48, 43, 39, 31, 25, 0]		 Costo: 86520.94977629452	 Delivery: 7640	 Pickup: 3306
Mezzo 5:	 Rotta: [0, 113, 150, 99, 58, 63, 100, 76, 142, 129, 82, 140, 110, 22, 20, 7, 28, 2, 15, 0]		 Costo: 93901.095437524	 Delivery: 5269	 Pickup: 2552
Mezzo 6:	 Rotta: [0, 53, 88, 137, 106, 122, 145, 66, 128, 136, 143, 127, 124, 35, 47, 3, 17, 38, 19, 0]		 Costo: 77062.64702041331	 Delivery: 7255	 Pickup: 2704
Mezzo 7:	 Rotta: [0, 69, 95, 1

In [41]:
print(f'numero nodi visitati, compreso il magazzino: {len(set_visited)}\nlista dei nodi: {set_visited}')

numero nodi visitati, compreso il magazzino: 151
lista dei nodi: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}


In [42]:
tot_cost=sum([x.cost for x in list_route])

In [43]:
path_solutions ='RPA_Solutions/Detailed_Solution_N6.txt'

round(((tot_cost * 100) / extract_total_cost_BP(path_solutions))-100 , 2)


85.34

In [44]:
list_route[3].route

[0,
 61,
 77,
 118,
 131,
 123,
 114,
 102,
 96,
 78,
 72,
 83,
 62,
 87,
 44,
 48,
 43,
 39,
 31,
 25,
 0]

In [45]:
for i in range(len(list_route[3].route)-1):
    G.add_edge(list_route[3].route[i],list_route[3].route[i+1])

In [46]:
plot = Plot(plot_width=500, plot_height=500,
            x_range=Range1d(-1.3, 3.3), y_range=Range1d(-1.3, 3.3))
plot.title.text = "Graph Interaction"

node_hover_tool = HoverTool(tooltips=[("Magazzino", "@index"),("Coordinate", '@pos')])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, pos, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine( line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

output_notebook()
show(plot)

Loading BokehJS ...